<a href="https://colab.research.google.com/github/41371131h-chi/114-1-/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [275]:
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
               gradio pandas beautifulsoup4 google-generativeai python-dateutil

In [276]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

In [277]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [278]:
from google.colab import userdata
api_key = userdata.get('HW3')
genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-2.5-flash')

In [279]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1nuhUMYejx4I0uq-2Fro6Bjlpd0H1VW0ecVyKXM1AePA/edit?usp=sharing"
WORKSHEET_NAME = "工作表3"
TIMEZONE = "Asia/Taipei"

In [280]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url(SHEET_URL)


# 從 gsheets 的 All-whiteboard-device 載入 sheets
sh = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sh[1:], columns=sh[0])
# 取得最前面的5筆資料
df.head()


""


In [281]:
def ensure_spreadsheet(name):
    try:
        sh = gc.open(name)  # returns gspread.models.Spreadsheet
    except gspread.SpreadsheetNotFound:
        sh = gc.create(name)
    return sh

sh = gsheets

In [282]:
def ensure_worksheet(sh, title, header):
    try:
        ws = sh.worksheet(title)
    except gspread.WorksheetNotFound:
        ws = sh.add_worksheet(title=title, rows="1000", cols=str(len(header)+5))
        ws.update([header])
    # 若沒有表頭就補上
    data = ws.get_all_values()
    if not data or (data and data[0] != header):
        ws.clear()
        ws.update([header])
    return ws

TASKS_HEADER = [
    "id","task","status","priority","est_min","start_time","end_time",
    "actual_min","pomodoros","due_date","labels","notes",
    "created_at","updated_at","completed_at","planned_for"
]
LOGS_HEADER = [
    "log_id","task_id","phase","start_ts","end_ts","minutes","cycles","note"
]
CLIPS_HEADER = ["clip_id","url","selector","text","href","created_at","added_to_task"]

ws_tasks = ensure_worksheet(sh, "tasks", TASKS_HEADER)
ws_logs  = ensure_worksheet(sh, "pomodoro_logs", LOGS_HEADER)
ws_clips = ensure_worksheet(sh, "web_clips", CLIPS_HEADER)


In [283]:
def tznow():
    return dt.now(gettz(TIMEZONE))

In [284]:
def write_df(ws, df, header):
    if df.empty:
        ws.clear()
        ws.update([header])
        return
    # 轉字串避免 gspread 型別問題
    df_out = df.copy()
    for c in df_out.columns:
        df_out[c] = df_out[c].astype(str)
    ws.clear()
    ws.update([header] + df_out.values.tolist())


In [285]:
def read_df(ws, header):
    df = get_as_dataframe(ws, evaluate_formulas=True, header=0)
    if df is None or df.empty:
        return pd.DataFrame(columns=header)
    df = df.fillna("")
    # 保證欄位齊全
    for c in header:
        if c not in df.columns:
            df[c] = ""
    # 型別微調
    if "est_min" in df.columns:
        df["est_min"] = pd.to_numeric(df["est_min"], errors="coerce").fillna(0).astype(int)
    if "actual_min" in df.columns:
        df["actual_min"] = pd.to_numeric(df["actual_min"], errors="coerce").fillna(0).astype(int)
    if "pomodoros" in df.columns:
        df["pomodoros"] = pd.to_numeric(df["pomodoros"], errors="coerce").fillna(0).astype(int)
    return df[header]

In [286]:
def refresh_all():
    return (
        read_df(ws_tasks, TASKS_HEADER).copy(),
        read_df(ws_logs, LOGS_HEADER).copy(),
    )

tasks_df, logs_df = refresh_all()
# 重新定義 _refresh

In [287]:
def refresh_all():
    return (
        read_df(ws_tasks, TASKS_HEADER).copy(),
        read_df(ws_logs, LOGS_HEADER).copy(),
    )

tasks_df, logs_df = refresh_all()
# 重新定義 _refresh

In [288]:
def add_task(task, priority, est_min, due_date, labels, notes, planned_for):
    global tasks_df
    _now = tznow().isoformat()
    new = pd.DataFrame([{
        "id": str(uuid.uuid4())[:8],
        "task": task.strip(),
        "status": "todo",
        "priority": priority or "M",
        "est_min": int(est_min) if est_min else 25,
        "start_time": "",
        "end_time": "",
        "actual_min": 0,
        "pomodoros": 0,
        "due_date": due_date or "",
        "labels": labels or "",
        "notes": notes or "",
        "created_at": _now,
        "updated_at": _now,
        "completed_at": "",
        "planned_for": planned_for or ""  # 可填 today / tomorrow / 空白
    }])
    tasks_df = pd.concat([tasks_df, new], ignore_index=True)
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return "✅ 已新增任務", tasks_df

In [289]:
def update_task_status(task_id, new_status):
    global tasks_df
    idx = tasks_df.index[tasks_df["id"] == task_id]
    if len(idx)==0:
        return "⚠️ 找不到任務", tasks_df
    i = idx[0]
    tasks_df.loc[i, "status"] = new_status
    tasks_df.loc[i, "updated_at"] = tznow().isoformat()
    if new_status == "done" and not tasks_df.loc[i, "completed_at"]:
        tasks_df.loc[i, "completed_at"] = tznow().isoformat()
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return "✅ 狀態已更新", tasks_df

In [290]:
def mark_done(task_id):
    return update_task_status(task_id, "done")

In [291]:
def recalc_task_actuals(task_id):
    """根據 logs_df 回寫 actual_min 與 pomodoros"""
    global tasks_df, logs_df
    work_logs = logs_df[(logs_df["task_id"]==task_id) & (logs_df["phase"]=="work")]
    total_min = work_logs["minutes"].astype(float).sum() if not work_logs.empty else 0
    pomos = int(round(total_min / 25.0))
    idx = tasks_df.index[tasks_df["id"]==task_id]
    if len(idx)==0: return
    i = idx[0]
    tasks_df.loc[i,"actual_min"] = int(total_min)
    tasks_df.loc[i,"pomodoros"] = pomos
    tasks_df.loc[i,"updated_at"] = tznow().isoformat()

In [292]:
def list_task_choices():
    global tasks_df
    if tasks_df.empty:
        return []
    # 顯示： [status] (P:priority) task  — id
    def row_label(r):
        return f"[{r['status']}] (P:{r['priority']}) {r['task']} — {r['id']}"
    return [(row_label(r), r["id"]) for _, r in tasks_df.iterrows()]

# 我們採「按鈕開始 / 結束」模式（避免後端阻塞），每次按「開始」會先記住 start_ts，
# 按「結束」時計算分鐘並寫入 logs，再回填任務 actual_min / pomodoros。

_active_sessions = {}  # { task_id: {"phase": "work"/"break", "start_ts": iso, "cycles": int} }

In [293]:
def start_phase(task_id, phase, cycles):
    if not task_id: return "⚠️ 請先選擇任務"
    _active_sessions[task_id] = {
        "phase": phase,
        "start_ts": tznow().isoformat(),
        "cycles": int(cycles) if cycles else 1
    }
    return f"▶️ 已開始：{phase}（task: {task_id}）"

In [294]:
def end_phase(task_id, note):
    global logs_df, tasks_df
    if task_id not in _active_sessions:
        return "⚠️ 尚未開始任何階段"
    sess = _active_sessions.pop(task_id)
    start = pd.to_datetime(sess["start_ts"])
    end = tznow()
    minutes = round((end - start).total_seconds() / 60.0, 2)
    log = pd.DataFrame([{
        "log_id": str(uuid.uuid4())[:8],
        "task_id": task_id,
        "phase": sess["phase"],
        "start_ts": start.isoformat(),
        "end_ts": end.isoformat(),
        "minutes": minutes,
        "cycles": int(sess["cycles"]),
        "note": note or ""
    }])
    logs_df = pd.concat([logs_df, log], ignore_index=True)
    write_df(ws_logs, logs_df, LOGS_HEADER)

    # 回填任務
    if sess["phase"] == "work":
        recalc_task_actuals(task_id)
        write_df(ws_tasks, tasks_df, TASKS_HEADER)

    return f"⏹️ 已結束：{sess['phase']}，紀錄 {minutes} 分鐘"


In [295]:
# AI 計畫（Gemini；無金鑰則規則式）
def generate_today_plan():
    global tasks_df
    # 以「due_date 是今天」或「planned_for = today」且不是 done 的任務為計畫清單
    today = tznow().date().isoformat()
    cand = tasks_df[
        ((tasks_df["due_date"]==today) | (tasks_df["planned_for"].str.lower()=="today")) &
        (tasks_df["status"]!="done")
    ].copy()
    if cand.empty:
        return "📭 今天沒有標記的任務。請在 Tasks 分頁把任務的 due_date 設為今天或 planned_for 設為 today。"

    # 先依 priority（H>M>L）+ est_min 排序
    pr_order = {"H":0, "M":1, "L":2}
    cand["p_ord"] = cand["priority"].map(pr_order).fillna(3)
    cand = cand.sort_values(["p_ord","est_min"], ascending=[True, True])

    # 嘗試 Gemini
    api_key = os.environ.get("GEMINI_API_KEY","").strip()
    if api_key:
        genai.configure(api_key=api_key)
        sys_prompt = (
            "你是一位任務規劃助理。請把輸入的任務（含估時與優先級）排成三段：morning、afternoon、evening，"
            "並給出每段的重點、順序、每項的時間預估與備註。總時數請大致符合任務估時總和。"
            "回傳以 Markdown 條列，格式：\n"
            "### Morning\n- [任務ID] 任務名稱（預估 xx 分）— 備註\n..."
            "### Afternoon\n...\n### Evening\n...\n"
        )
        items = []
        for _, r in cand.iterrows():
            items.append({
                "id": r["id"], "task": r["task"], "est_min": int(r["est_min"]),
                "priority": r["priority"]
            })
        user_content = json.dumps({"today": today, "tasks": items}, ensure_ascii=False)
        try:
            model = genai.GenerativeModel("gemini-1.5-flash")
            resp = model.generate_content(sys_prompt + "\n\n" + user_content)
            plan_md = resp.text
        except Exception as e:
            plan_md = f"⚠️ Gemini 失敗：{e}\n\n改用規則式規劃。"
    else:
        plan_md = "🔧 未設定 GEMINI_API_KEY，使用規則式規劃。\n\n"

    # 規則式：把高優先任務平均切到上午/下午/晚上
    buckets = {"morning": [], "afternoon": [], "evening": []}
    total = len(cand)
    for i, (_, r) in enumerate(cand.iterrows()):
        if i % 3 == 0:
            buckets["morning"].append(r)
        elif i % 3 == 1:
            buckets["afternoon"].append(r)
        else:
            buckets["evening"].append(r)

    def sec_md(name, rows):
        if not rows: return f"### {name.title()}\n（無）\n"
        lines = [f"### {name.title()}"]
        for r in rows:
            lines.append(f"- [{r['id']}] {r['task']}（預估 {int(r['est_min'])} 分，P:{r['priority']}）")
        return "\n".join(lines) + "\n"

    rule_md = sec_md("morning", buckets["morning"]) + "\n" + \
              sec_md("afternoon", buckets["afternoon"]) + "\n" + \
              sec_md("evening", buckets["evening"])

    return (plan_md + "\n---\n" + rule_md).strip()

In [296]:
# 今日完成率
def today_summary():
    global tasks_df
    today = tznow().date().isoformat()
    planned = tasks_df[
        ((tasks_df["due_date"]==today) | (tasks_df["planned_for"].str.lower()=="today"))
    ]
    done = planned[planned["status"]=="done"]
    total = len(planned)
    done_n = len(done)
    rate = (done_n/total*100) if total>0 else 0
    return f"📅 今日計畫任務：{total}；✅ 完成：{done_n}；📈 完成率：{rate:.1f}%"

In [297]:
def add_clips_as_tasks(clip_ids, default_priority, est_min):
    global clips_df, tasks_df
    if not clip_ids:
        return "⚠️ 請先勾選要加入的爬蟲項目", clips_df, tasks_df
    sel = clips_df[clips_df["clip_id"].isin(clip_ids)]
    _now = tznow().isoformat()
    new_tasks = []
    for _, r in sel.iterrows():
        title = r["text"] or r["href"] or "（未命名）"
        note = f"來源：{r['url']}\n選擇器：{r['selector']}\n連結：{r['href']}"
        new_tasks.append({
            "id": str(uuid.uuid4())[:8],
            "task": title[:120],
            "status": "todo",
            "priority": default_priority or "M",
            "est_min": int(est_min) if est_min else 25,
            "start_time": "",
            "end_time": "",
            "actual_min": 0,
            "pomodoros": 0,
            "due_date": "",
            "labels": "from:crawler",
            "notes": note,
            "created_at": _now,
            "updated_at": _now,
            "completed_at": "",
            "planned_for": ""
        })
    if new_tasks:
        tasks_df = pd.concat([tasks_df, pd.DataFrame(new_tasks)], ignore_index=True)
        # 標記已加入
        clips_df.loc[clips_df["clip_id"].isin(clip_ids), "added_to_task"] = "yes"
        write_df(ws_tasks, tasks_df, TASKS_HEADER)
        write_df(ws_clips, clips_df, CLIPS_HEADER)
        return f"✅ 已加入 {len(new_tasks)} 項為任務", clips_df, tasks_df
    return "⚠️ 無可加入項目", clips_df, tasks_df

In [298]:
def _refresh():
    global tasks_df, logs_df
    tasks_df, logs_df = refresh_all()

    # 準備輸出
    task_choices = list_task_choices()
    summary_text = today_summary()

    # 返回時，明確複製 DFs，確保 Gradio 認為它們是新的值，強制更新 DataGrid
    return (
        tasks_df.copy(),        # 1. grid_tasks
        logs_df.copy(),         # 2. grid_logs
        task_choices,           # 3. task_choice (Tasks 更新狀態)
        summary_text,           # 4. out_summary (頂部摘要)
        summary_text,           # 5. out_summary2 (Summary 頁面摘要)
        task_choices,           # 6. sel_task (Pomodoro 選單)
        task_choices,           # 7. task_to_delete (Delete 選單)
        task_choices            # 8. task_choice (冗餘，但為匹配 Gradio 綁定)
    )

In [299]:
# -----------------------------------------------------------
# 刪除任務 (Delete Task)
# -----------------------------------------------------------
def delete_task(task_id):
    global tasks_df
    idx = tasks_df.index[tasks_df["id"] == task_id]
    if len(idx) == 0:
        return "⚠️ 找不到任務", tasks_df.copy()

    # 刪除該行
    tasks_df = tasks_df.drop(idx).reset_index(drop=True)

    # 寫回 Google Sheet
    write_df(ws_tasks, tasks_df, TASKS_HEADER)

    return f"✅ 任務 {task_id} 已刪除", tasks_df.copy()

In [300]:
# -----------------------------------------------------------
# 查詢強化 (Query Tasks)
# -----------------------------------------------------------
def query_tasks(status_filter, start_date, end_date):
    """根據狀態和時間範圍篩選任務"""
    filtered_df = tasks_df.copy()

    # 1. 狀態篩選
    if status_filter and status_filter != "all":
        filtered_df = filtered_df[filtered_df["status"] == status_filter]

    # 2. 時間範圍篩選 (針對 completed_at)
    if start_date or end_date:
        filtered_df['completed_at_dt'] = pd.to_datetime(
            filtered_df['completed_at'], errors='coerce', utc=True
        ).dt.tz_convert(gettz(TIMEZONE))

        # 處理開始日期
        if start_date:
            start_dt = pd.to_datetime(start_date).tz_localize(gettz(TIMEZONE)).normalize()
            filtered_df = filtered_df[filtered_df['completed_at_dt'] >= start_dt]

        # 處理結束日期
        if end_date:
            end_dt = pd.to_datetime(end_date).tz_localize(gettz(TIMEZONE)).normalize() + timedelta(days=1)
            filtered_df = filtered_df[filtered_df['completed_at_dt'] < end_dt]

        filtered_df = filtered_df.drop(columns=['completed_at_dt'])

    # 限制欄位顯示
    display_cols = ["id","task","status","priority","est_min","actual_min","completed_at"]
    return filtered_df[display_cols]

In [301]:
import io # 需要 io 函式庫來處理字串 I/O

# -----------------------------------------------------------
# 匯出資料 (Export Data)
# -----------------------------------------------------------
def export_data(format_type):
    """將任務資料匯出為 CSV 或 JSON 格式字串"""
    if tasks_df.empty:
        return None, "⚠️ 任務清單為空，無法匯出"

    # 清理不必要的欄位，如 created_at/updated_at 等，以簡化匯出
    export_cols = [c for c in tasks_df.columns if c not in ["created_at","updated_at","completed_at"]]
    df_out = tasks_df[export_cols].copy()

    if format_type == "csv":
        # 使用 io.StringIO 儲存為字串，模擬檔案內容
        csv_buffer = io.StringIO()
        df_out.to_csv(csv_buffer, index=False, encoding='utf-8')
        return csv_buffer.getvalue(), f"✅ 已匯出 {len(df_out)} 筆任務 (CSV)"

    elif format_type == "json":
        json_str = df_out.to_json(orient='records', indent=2, force_ascii=False)
        return json_str, f"✅ 已匯出 {len(df_out)} 筆任務 (JSON)"

    return None, "⚠️ 不支援的匯出格式"

In [302]:
# -----------------------------------------------------------
# 匯入資料 (Import Data)
# -----------------------------------------------------------
def import_data(file_obj, format_type):
    """從上傳的 CSV/JSON 檔案匯入資料"""
    global tasks_df

    if file_obj is None:
        return tasks_df.copy(), "⚠️ 請先上傳檔案"

    try:
        if format_type == "csv":
            new_df = pd.read_csv(file_obj.name)
        elif format_type == "json":
            new_df = pd.read_json(file_obj.name)
        else:
            return tasks_df.copy(), "⚠️ 不支援的匯入格式"

        # 確保新舊 DF 欄位一致
        missing_cols = [c for c in TASKS_HEADER if c not in new_df.columns]
        for c in missing_cols:
            new_df[c] = ""

        # 覆蓋或合併 (這裡採用覆蓋所有欄位，但保留原有 ID)
        new_df = new_df[TASKS_HEADER]

        # 確保 ID 唯一，沒有 ID 的產生新 ID，有衝突的跳過（簡化處理）
        new_df['id'] = new_df['id'].apply(lambda x: str(uuid.uuid4())[:8] if not x else x)

        # 寫入邏輯：這裡假設是覆蓋式匯入 (即用新資料取代舊資料)
        tasks_df = new_df.copy()
        write_df(ws_tasks, tasks_df, TASKS_HEADER)

        return tasks_df.copy(), f"✅ 成功匯入 {len(new_df)} 筆任務並已寫回 Sheet"

    except Exception as e:
        return tasks_df.copy(), f"❌ 匯入失敗：{e}"

In [303]:
import plotly.express as px
import plotly.graph_objects as go
# Gradio 需要 plotly 的 figure 作為輸出

def get_viz_data():
    """計算視覺化所需的統計數據"""
    if tasks_df.empty:
        return None, None

    df_viz = tasks_df.copy()

    # 確保 est_min 和 actual_min 是數字
    df_viz["est_min"] = pd.to_numeric(df_viz["est_min"], errors="coerce").fillna(0)
    df_viz["actual_min"] = pd.to_numeric(df_viz["actual_min"], errors="coerce").fillna(0)

    # 任務狀態計數
    status_counts = df_viz.groupby("status").size().reset_index(name="count")

    # 優先級時間統計
    time_stats = df_viz.groupby("priority")[["est_min", "actual_min"]].sum().reset_index()
    time_stats = time_stats.melt(
        id_vars="priority",
        value_vars=["est_min", "actual_min"],
        var_name="Type",
        value_name="Minutes"
    )
    return status_counts, time_stats

def create_status_pie_chart():
    """創建任務狀態圓餅圖"""
    status_counts, _ = get_viz_data()
    if status_counts is None:
        return go.Figure().add_annotation(text="無任務數據", showarrow=False)

    fig = px.pie(
        status_counts,
        values='count',
        names='status',
        title='任務狀態分佈',
        color='status',
        color_discrete_map={
             'todo':'#EF553B', 'in-progress':'#00CC96', 'done':'#636EFA'
        }
    )
    return fig

def create_time_bar_chart():
    """創建優先級時間柱狀圖（預估 vs 實際）"""
    _, time_stats = get_viz_data()
    if time_stats is None:
        # 🚨 修正：確保在數據為空時返回一個 Plotly Figure
        return go.Figure().add_annotation(text="無任務數據", showarrow=False)

    fig = px.bar(
        time_stats,
        x="priority",
        y="Minutes",
        color="Type",
        barmode="group",
        title="優先級別時間統計 (預估 vs 實際)",
        labels={"priority": "優先級", "Minutes": "總分鐘數"},
        category_orders={"priority": ["H", "M", "L"]}
    )
    return fig

In [304]:
print(tasks_df["est_min"].dtype)
print(tasks_df["actual_min"].dtype)

int64
int64


In [305]:
# =========================
# Gradio 介面
# =========================

with gr.Blocks(title="待辦清單＋番茄鐘＋AI 計畫 (升級版)") as demo:
    gr.Markdown("# ✅ 待辦清單與番茄鐘 (Google Sheet＋Gradio＋AI Planner)")
    with gr.Row():
        # 移除 clips_df，更新 _refresh 綁定
        btn_refresh = gr.Button("🔄 重新整理（Sheet → App）")
        out_summary = gr.Markdown(today_summary())

    with gr.Tab("Tasks"):
        with gr.Row():
            with gr.Column(scale=2):
                task = gr.Textbox(label="任務名稱", placeholder="寫 HW3 報告 / 修正 SQL / …")
                priority = gr.Dropdown(["H","M","L"], value="M", label="優先級")
                est_min = gr.Number(value=25, label="預估時間（分鐘）", precision=0)
                due_date = gr.Textbox(label="到期日（YYYY-MM-DD，可空白）")
                labels = gr.Textbox(label="標籤（逗號分隔，可空白）")
                notes = gr.Textbox(label="備註（可空白）")
                planned_for = gr.Dropdown(["","today","tomorrow"], value="", label="規劃歸屬")
                btn_add = gr.Button("➕ 新增任務")
                msg_add = gr.Markdown()
            with gr.Column(scale=3):
                grid_tasks = gr.Dataframe(value=tasks_df, label="任務清單（直接從 Sheet 來）", interactive=False)

        with gr.Row():
            task_choice = gr.Dropdown(choices=list_task_choices(), label="選取任務（用於更新）")
            new_status = gr.Dropdown(["todo","in-progress","done"], value="in-progress", label="更新狀態")
            btn_update = gr.Button("✏️ 更新狀態")
            btn_done = gr.Button("✅ 直接標記完成")
            msg_update = gr.Markdown()
        gr.Markdown("---")
        gr.Markdown("### 🗑️ 刪除任務")
        with gr.Row():
            task_to_delete = gr.Dropdown(choices=list_task_choices(), label="選取要刪除的任務")
            btn_delete = gr.Button("❌ 刪除選取的任務")
            msg_delete = gr.Markdown()

    with gr.Tab("Pomodoro"):
        with gr.Row():
            sel_task = gr.Dropdown(choices=list_task_choices(), label="選擇任務")
            cycles = gr.Number(value=1, precision=0, label="番茄數（僅作紀錄）")
        with gr.Row():
            btn_start_work = gr.Button("▶️ 開始工作")
            note_work = gr.Textbox(label="工作備註（可空白）")
            btn_end_work = gr.Button("⏹️ 結束工作並記錄")
        with gr.Row():
            btn_start_break = gr.Button("🍵 開始休息")
            note_break = gr.Textbox(label="休息備註（可空白）")
            btn_end_break = gr.Button("⏹️ 結束休息並記錄")
        msg_pomo = gr.Markdown()
        grid_logs = gr.Dataframe(value=logs_df, label="番茄鐘紀錄", interactive=False)

    with gr.Tab("AI Plan"):
        gr.Markdown("把**今天的任務**排成 **morning / afternoon / evening** 三段行動計畫。若未設 GEMINI_API_KEY，會用規則式。")
        btn_plan = gr.Button("🧠 產生今日計畫")
        out_plan = gr.Markdown()

    with gr.Tab("Query"):
        gr.Markdown("### 🔍 任務查詢")
        with gr.Row():
            q_status = gr.Radio(["all", "todo", "in-progress", "done"], value="all", label="狀態篩選")
            q_start_date = gr.Textbox(label="完成起始日期 (YYYY-MM-DD)", placeholder="2025-01-01")
            q_end_date = gr.Textbox(label="完成結束日期 (YYYY-MM-DD)", placeholder="2025-12-31")
            btn_query = gr.Button("執行查詢")
        grid_query = gr.Dataframe(label="查詢結果", interactive=False)

    with gr.Tab("Summary"):
        btn_summary = gr.Button("📊 重新計算今日完成率")
        out_summary2 = gr.Markdown()

    with gr.Tab("Data Management"):
        gr.Markdown("### 📥 匯入資料 (Import)")
        with gr.Row():
            file_to_import = gr.File(label="上傳 CSV/JSON 任務檔案")
            import_format = gr.Radio(["csv", "json"], value="csv", label="檔案格式")
            btn_import = gr.Button("⬆️ 匯入並更新 Sheet")
        msg_import = gr.Markdown()

        gr.Markdown("### 📤 匯出資料 (Export to CSV/JSON)")
        with gr.Row():
            export_format = gr.Radio(["csv", "json"], value="csv", label="檔案格式")
            btn_export = gr.Button("⬇️ 匯出任務清單")
            file_export = gr.File(label="下載檔案")
        msg_export = gr.Markdown()

    with gr.Tab("Visualization"):
        gr.Markdown("### 📊 統計圖表")
        plot_status = gr.Plot(label="任務狀態分佈")
        plot_time = gr.Plot(label="優先級時間統計")
        btn_viz = gr.Button("📈 產生視覺化圖表")

    with gr.Tab("Summary"):
        btn_summary = gr.Button("📊 重新計算今日完成率")
        out_summary2 = gr.Markdown()

    # === 綁定動作 ===
    # 修改後（注意 _refresh 也要修改）：
    btn_refresh.click(
         _refresh,
         outputs=[
             grid_tasks,# 1. tasks_df.copy()
             grid_logs,# 2. logs_df.copy()
             task_choice,# 3. task_choices (Tasks 更新)
             out_summary,# 4. summary_text (頂部摘要)
             out_summary2,# 5. summary_text (Summary 頁面)
             sel_task,# 6. task_choices (Pomodoro)
             task_to_delete,# 7. task_choices (Delete)
             task_choice# 8. task_choices (冗餘，但為匹配)
             ]
         )

    btn_add.click(
        add_task,
        inputs=[task, priority, est_min, due_date, labels, notes, planned_for],
        outputs=[msg_add, grid_tasks]
    )

    btn_update.click(
        update_task_status,
        inputs=[task_choice, new_status],
        outputs=[msg_update, grid_tasks]
    )

    btn_done.click(
        mark_done,
        inputs=[task_choice],
        outputs=[msg_update, grid_tasks]
    )

    btn_start_work.click(
        start_phase, inputs=[sel_task, gr.State("work"), cycles], outputs=[msg_pomo]
    )
    btn_end_work.click(
        end_phase, inputs=[sel_task, note_work], outputs=[msg_pomo]
    )
    btn_start_break.click(
        start_phase, inputs=[sel_task, gr.State("break"), cycles], outputs=[msg_pomo]
    )
    btn_end_break.click(
        end_phase, inputs=[sel_task, note_break], outputs=[msg_pomo]
    )

    btn_plan.click(generate_today_plan, outputs=[out_plan])

    # 綁定新的刪除功能
    btn_delete.click(
        delete_task,
        inputs=[task_to_delete],
        outputs=[msg_delete, grid_tasks]
    )

    # 綁定查詢功能
    btn_query.click(
        query_tasks,
        inputs=[q_status, q_start_date, q_end_date],
        outputs=[grid_query]
    )

    # 綁定匯出功能
    btn_export.click(
        export_data,
        inputs=[export_format],
        outputs=[file_export, msg_export],
        api_name="export"
    )

    # 綁定匯入功能
    btn_import.click(
        import_data,
        inputs=[file_to_import, import_format],
        outputs=[grid_tasks, msg_import]
    )

    # 綁定視覺化功能
    btn_viz.click(
        lambda: [create_status_pie_chart(), create_time_bar_chart()],
        outputs=[plot_status, plot_time]
    )

    btn_summary.click(today_summary, outputs=[out_summary2])

In [306]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4b5efd2336a53a97ea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
